In [1]:
print('hi')

hi


In [2]:
import tensorflow_hub as hub
from bert_serving.client import BertClient
bc = BertClient()

In [3]:
import datahelper
datahelper = datahelper.DataHelper(embedding_path="../embedding/STCWiki/STCWiki_mincount0.model.bin")

Using TensorFlow backend.


In [4]:
REMOVE_STOPWORDS = False
TO_LOWER = True
TOKEN_TYPE = 'nltk'
EMB = 'stc' # glove or stc

In [5]:
trainX, trainX_bert, trainND, trainDQ, train_turns, train_masks = datahelper.get_model_train_data(
    'train',
    TOKEN_TYPE, 
    REMOVE_STOPWORDS, 
    TO_LOWER,
    EMB,
    bert=True,
)

# devX, devX_bert, devND, devDQ, dev_turns, dev_masks = datahelper.get_model_train_data(
#     'dev',
#     TOKEN_TYPE, 
#     REMOVE_STOPWORDS, 
#     TO_LOWER,
#     EMB,
#     bert=True,
# )

# testX, testX_bert, test_turns, test_masks = datahelper.get_model_test_data(
#     TOKEN_TYPE, 
#     REMOVE_STOPWORDS, 
#     TO_LOWER,
#     EMB,
#     bert=True,
# )

/usr/local/lib/python3.5/dist-packages/bert_serving/client/__init__.py:285: UserWarning: some of your sentences have more tokens than "max_seq_len=150" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [6]:
trainX.shape, trainX_bert.shape

((1337, 7, 150, 100), (1337, 7, 1024))

In [19]:
devX.shape, devX_bert.shape

((335, 7, 150, 100), (335, 7, 1024))

In [7]:
import pickle
pickle.dump(trainX_bert, open('trainX_bert.p', 'wb'))
# pickle.dump(devX_bert, open('devX_bert.p', 'wb'))
# pickle.dump(testX_bert, open('testX_bert.p', 'wb'))

In [3]:
trainX = pickle.load(open('trainX_bert.p', 'rb'))
devX = pickle.load(open('devX_bert.p', 'rb'))
testX = pickle.load(open('testX_bert.p', 'rb'))

EOFError: Ran out of input

In [22]:
devX_berts.shape, devX_bert.shape

((335, 7, 1024), (335, 7, 1024))

In [11]:
a = bc.encode([['hi', 'he']], is_tokenized=True)

In [12]:
a

array([[-0.30754888, -0.17501961, -0.2558127 , ..., -0.37166625,
         0.12095191,  0.13405366]], dtype=float32)

In [15]:
# import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [16]:
# bert_module = hub.Module("https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", trainable=False)